In [42]:
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure
from skimage.metrics import structural_similarity 
from sklearn.metrics.pairwise import cosine_similarity
import logging

In [43]:

def crop_to_common_size(image1, image2):
    # Find the minimum dimensions
    min_height = min(image1.shape[0], image2.shape[0])
    min_width = min(image1.shape[1], image2.shape[1])

    # Crop both images to the minimum dimensions
    cropped_image1 = image1[:min_height, :min_width]
    cropped_image2 = image2[:min_height, :min_width]

    return cropped_image1, cropped_image2

# Configuring the logging module
logging.basicConfig(filename='image_processing.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Bamboo and Birch Distance (BBD)

def calculate_bbd(image1, image2):
    # Ensure the images have the same size
    if image1.shape != image2.shape:
        raise ValueError("Images must have the same dimensions")

    # Convert images to grayscale if needed
    if len(image1.shape) == 3:
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Calculate BBD distance
    bbd_distance = np.sum(np.abs(image1 - image2)) / float(image1.size)

    return bbd_distance
# Histogram of Oriented Gradients (HOG)
def calculate_hog(image):
    # Convert the image to grayscale if it's in color
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate HOG features
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True)

    # Rescale histogram for better display
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    return fd, hog_image_rescaled

def compare_hog_features(fd1, fd2):
    # Reshape feature vectors for cosine similarity calculation
    fd1 = fd1.reshape(1, -1)
    fd2 = fd2.reshape(1, -1)

    # Calculate cosine similarity between HOG feature vectors
    similarity = cosine_similarity(fd1, fd2)[0][0]

    return similarity

# Structural Similarity Index (SSI)
def calculate_ssi(image1, image2):
    # Convert images to grayscale if needed
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Calculate SSIM
    ssi_index, _ = ssim(image1_gray, image2_gray, full=True)

    return ssi_index

# Color Histograms
def calculate_color_histogram(image):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Calculate histogram
    hist = cv2.calcHist([hsv_image], [0, 1], None, [180, 256], [0, 180, 0, 256])

    # Normalize histogram
    hist = cv2.normalize(hist, hist)

    return hist

# Structural Comparison
def compare_images_ssim(image1, image2):
    # Convert images to grayscale if needed
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Calculate SSIM using OpenCV
    ssim_index, _ = structural_similarity(gray1, gray2, full=True)

    return ssim_index

# Example usage:
# try:
image1_in = cv2.imread('./samediff-apples.jpg')
image2_in = cv2.imread('./image3.png')
image1, image2 = crop_to_common_size(image1_in, image2_in)
# BBD
bbd_distance = calculate_bbd(image1, image2)
logging.info(f"BBD Distance: {bbd_distance}")

# HOG
hog_features, hog_image = calculate_hog(image1)
logging.info("HOG Calculation Successful")

# SSI
ssi_value = calculate_ssi(image1, image2)
logging.info(f"SSI Value: {ssi_value}")

# Color Histogram
color_histogram = calculate_color_histogram(image1)
logging.info("Color Histogram Calculation Successful")

# SSIM
ssim_value = compare_images_ssim(image1, image2)
logging.info(f"SSIM Value: {ssim_value}")

# except Exception as e:
#     logging.error(f"Error: {str(e)}")
